<a href="https://colab.research.google.com/github/smjason0502/python-deep-learning-pytorch/blob/main/Part_04_%EB%B9%88%EC%B9%B8_%EC%B6%94%EA%B0%80%EB%AC%B8%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 대용량의 데이터를 이용해 학습이 완료된 모델을 적은 수의 데이터에 맞게 Fine-tuning하는 Transfer Learning를 GoogLeNet으로 실습해보기

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, datasets

if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')

print('Using PyTorch version:', torch.__version__, ' Device:', DEVICE)

BATCH_SIZE = 32
EPOCHS = 10

!wget --no-check-certificate https://download.pytorch.org/tutorial/hymenoptera_data.zip

import zipfile

zip_file = 'hymenoptera_data.zip'
zip_ref = zipfile.ZipFile(zip_file, 'r')
zip_ref.extractall('./data')
zip_ref.close()

!rm hymenoptera_data.zip

#####################################################################################################
## 문제 1. 다음과 같은 요구 사항을 만족하는 이미지 전처리 파이프라인을 구성하는 코드를 작성하세요. ##
## < 요구사항 >                                                                                    ##
## (1) 훈련 데이터 : 이미지를 무작위로 자르고 크기를 224x224로 조정 & 이미지를 좌우로 무작위 반전  ##
##                 & 각 채널(빨강, 초록, 파랑)의 평균과 표준편차를 [0.5, 0.5, 0.5]로 정규화        ##
## (2) 검증 데이터 : 이미지를 가운데 부분만 자르고 크기를 224x224로 설정 & 이미지의 크기는 256x256 ##
##                 & 각 채널(빨강, 초록, 파랑)의 평균과 표준편차를 [0.5, 0.5, 0.5]로 정규화        ##
#####################################################################################################

#####################################################################################################







#####################################################################################################


data_dir = 'data/hymenoptera_data'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
           data_transforms[x]) for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x],
            batch_size = BATCH_SIZE,
            num_workers = 0,
            shuffle = True) for x in ['train', 'val']}

##############################################################################################################################################
## 문제 2. 다음의 예시처럼 나오도록 모델 학습 및 평가 함수를 작성하세요.                                                                    ##
## < 출력 예시 >                                                                                                                            ##
## Train Epoch: 1 [1280/60000 (2%)]    Train Loss: 0.645321                                                                                 ##
## Train Epoch: 1 [2560/60000 (4%)]    Train Loss: 0.612456                                                                                 ##
## ...                                                                                                                                      ##
## [EPOCH: 1],    Test Loss: 0.5123,    Test Accuracy: 82.45%                                                                               ##
##                                                                                                                                          ##
## < 힌트 >                                                                                                                                 ##
## 1. model.train()과 model.eval()을 통해 모델의 모드를 설정할 수 있습니다.                                                                 ##
## 2. 모델이 여러 출력을 반환하는 경우, 첫 번째 출력만 사용하여 손실을 계산하세요. [if isinstance(outputs, tuple)를 사용하세요]             ##
## 3. 학습할 때는 옵티마이저의 기울기를 초기화하고, backward 함수를 통해 기울기를 계산한 후 옵티마이저로 가중치를 업데이트하세요.           ##
## 4. 평가할 때는 torch.no_grad()를 사용하여 기울기를 계산하지 않도록 설정하세요.                                                           ##
## 5. 특정 간격마다 학습 상태를 출력하세요.                                                                                                 ##
##############################################################################################################################################

##############################################################################################################################################







##############################################################################################################################################

import torchvision.models as models
model = models.googlenet(pretrained=False).cuda()  # GoogLeNet instead of ResNet18
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)
model = model.cuda()

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()

print(model)

print('미리 학습되지 않은 GoogLeNet 학습을 실행하며 Test set의 Loss와 Accuracy 확인하기 \n')

for epoch in range(1, EPOCHS + 1):
    train(model, dataloaders["train"], optimizer, log_interval=5)
    test_loss, test_accuracy = evaluate(model, dataloaders["val"])
    print('\n[EPOCH: {}], \tTest Loss: {:.4f}, \tTest Accuracy: {:.2f} % \n'.format(epoch, test_loss, test_accuracy))

model = models.googlenet(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)
model = model.cuda()

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

for epoch in range(1, EPOCHS + 1):
    train(model, dataloaders["train"], optimizer, log_interval=5)
    valid_loss, valid_accuracy = evaluate(model, dataloaders["val"])
    print("\n[EPOCH: {}], \tTest Loss: {:.4f}, \tTest Accuracy: {:.2f}% \n".format(epoch, valid_loss, valid_accuracy))

Using PyTorch version: 2.4.1+cu121  Device: cuda
--2024-09-28 11:10:18--  https://download.pytorch.org/tutorial/hymenoptera_data.zip
Resolving download.pytorch.org (download.pytorch.org)... 3.165.102.31, 3.165.102.113, 3.165.102.36, ...
Connecting to download.pytorch.org (download.pytorch.org)|3.165.102.31|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 47286322 (45M) [application/zip]
Saving to: ‘hymenoptera_data.zip’

hymenoptera_data.zi 100%[===================>]  45.10M   238MB/s    in 0.2s    

2024-09-28 11:10:19 (238 MB/s) - ‘hymenoptera_data.zip’ saved [47286322/47286322]



/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/usr/local/lib/python3.10/dist-packages/torchvision/models/googlenet.py:47: FutureWarning: The default weight initialization of GoogleNet will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(


GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=GoogLeNet_Weights.IMAGENET1K_V1`. You can also use `weights=GoogLeNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Train Epoch: 1 [0/244 (0%)]	Train Loss: 0.694229
Train Epoch: 1 [160/244 (62%)]	Train Loss: 0.524859

[EPOCH: 1], 	Test Loss: 0.0146, 	Test Accuracy: 93.46% 

Train Epoch: 2 [0/244 (0%)]	Train Loss: 0.473990
Train Epoch: 2 [160/244 (62%)]	Train Loss: 0.356648

[EPOCH: 2], 	Test Loss: 0.0102, 	Test Accuracy: 92.81% 

Train Epoch: 3 [0/244 (0%)]	Train Loss: 0.289889
Train Epoch: 3 [160/244 (62%)]	Train Loss: 0.255627

[EPOCH: 3], 	Test Loss: 0.0078, 	Test Accuracy: 94.12% 

Train Epoch: 4 [0/244 (0%)]	Train Loss: 0.242904
Train Epoch: 4 [160/244 (62%)]	Train Loss: 0.197370

[EPOCH: 4], 	Test Loss: 0.0066, 	Test Accuracy: 94.77% 

Train Epoch: 5 [0/244 (0%)]	Train Loss: 0.209389
Train Epoch: 5 [160/244 (62%)]	Train Loss: 0.126594

[EPOCH: 5], 	Test Loss: 0.0058, 	Test Accuracy: 94.12% 

Train Epoch: 6 [0/244 (0%)]	Train Loss: 0.118441
Train Epoch: 6 [160/244 (62%)]	Train Loss: 0.119298

[EPOCH: 6], 	Test Loss: 0.0060, 	Test Accuracy: 94.12% 

Train Epoch: 7 [0/244 (0%)]	Train Loss: 0.1339

# 해답지

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, datasets

if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')

print('Using PyTorch version:', torch.__version__, ' Device:', DEVICE)

BATCH_SIZE = 32
EPOCHS = 10

!wget --no-check-certificate https://download.pytorch.org/tutorial/hymenoptera_data.zip

import zipfile

zip_file = 'hymenoptera_data.zip'
zip_ref = zipfile.ZipFile(zip_file, 'r')
zip_ref.extractall('./data')
zip_ref.close()

!rm hymenoptera_data.zip

#####################################################################################################
## 문제 1. 다음과 같은 요구 사항을 만족하는 이미지 전처리 파이프라인을 구성하는 코드를 작성하세요. ##
## < 요구사항 >                                                                                    ##
## (1) 훈련 데이터 : 이미지를 무작위로 자르고 크기를 224x224로 조정 & 이미지를 좌우로 무작위 반전  ##
##                 & 각 채널(빨강, 초록, 파랑)의 평균과 표준편차를 [0.5, 0.5, 0.5]로 정규화        ##
## (2) 검증 데이터 : 이미지를 가운데 부분만 자르고 크기를 224x224로 설정 & 이미지의 크기는 256x256 ##
##                 & 각 채널(빨강, 초록, 파랑)의 평균과 표준편차를 [0.5, 0.5, 0.5]로 정규화        ##
#####################################################################################################

#####################################################################################################
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ]),
    'val': transforms.Compose([
        transforms.CenterCrop(224),
        transforms.Resize(256),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ]),
}
#####################################################################################################

data_dir = 'data/hymenoptera_data'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
           data_transforms[x]) for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x],
            batch_size = BATCH_SIZE,
            num_workers = 0,
            shuffle = True) for x in ['train', 'val']}

##############################################################################################################################################
## 문제 2. 다음의 예시처럼 나오도록 모델 학습 및 평가 함수를 작성하세요.                                                                    ##
## < 출력 예시 >                                                                                                                            ##
## Train Epoch: 1 [1280/60000 (2%)]    Train Loss: 0.645321                                                                                 ##
## Train Epoch: 1 [2560/60000 (4%)]    Train Loss: 0.612456                                                                                 ##
## ...                                                                                                                                      ##
## [EPOCH: 1],    Test Loss: 0.5123,    Test Accuracy: 82.45%                                                                               ##
##                                                                                                                                          ##
## < 힌트 >                                                                                                                                 ##
## 1. model.train()과 model.eval()을 통해 모델의 모드를 설정할 수 있습니다.                                                                 ##
## 2. 모델이 여러 출력을 반환하는 경우, 첫 번째 출력만 사용하여 손실을 계산하세요. [if isinstance(outputs, tuple)를 사용하세요]             ##
## 3. 학습할 때는 옵티마이저의 기울기를 초기화하고, backward 함수를 통해 기울기를 계산한 후 옵티마이저로 가중치를 업데이트하세요.           ##
## 4. 평가할 때는 torch.no_grad()를 사용하여 기울기를 계산하지 않도록 설정하세요.                                                           ##
## 5. 특정 간격마다 학습 상태를 출력하세요.                                                                                                 ##
##############################################################################################################################################

##############################################################################################################################################
def train(model, train_loader, optimizer, log_interval):
    model.train()
    for batch_idx, (image, label) in enumerate(train_loader):
        image = image.to(DEVICE)
        label = label.to(DEVICE)
        optimizer.zero_grad()

        outputs = model(image)
        if isinstance(outputs, tuple):
            output = outputs[0]
        else:
            output = outputs

        loss = criterion(output, label)
        loss.backward()
        optimizer.step()

        if batch_idx % log_interval == 0:
            print("Train Epoch: {} [{}/{} ({:.0f}%)]\tTrain Loss: {:.6f}".format(
                epoch, batch_idx * len(image),
                len(train_loader.dataset), 100. * batch_idx / len(train_loader),
                loss.item()))

def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0

    with torch.no_grad():
        for image, label in test_loader:
            image = image.to(DEVICE)
            label = label.to(DEVICE)

            outputs = model(image)
            if isinstance(outputs, tuple):
                output = outputs[0]
            else:
                output = outputs

            test_loss += criterion(output, label).item()
            prediction = output.max(1, keepdim=True)[1]
            correct += prediction.eq(label.view_as(prediction)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy
##############################################################################################################################################

import torchvision.models as models
model = models.googlenet(pretrained=False).cuda()  # GoogLeNet instead of ResNet18
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)
model = model.cuda()

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()

print(model)

print('미리 학습되지 않은 GoogLeNet 학습을 실행하며 Test set의 Loss와 Accuracy 확인하기 \n')

for epoch in range(1, EPOCHS + 1):
    train(model, dataloaders["train"], optimizer, log_interval=5)
    test_loss, test_accuracy = evaluate(model, dataloaders["val"])
    print('\n[EPOCH: {}], \tTest Loss: {:.4f}, \tTest Accuracy: {:.2f} % \n'.format(epoch, test_loss, test_accuracy))

model = models.googlenet(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)
model = model.cuda()

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

for epoch in range(1, EPOCHS + 1):
    train(model, dataloaders["train"], optimizer, log_interval=5)
    valid_loss, valid_accuracy = evaluate(model, dataloaders["val"])
    print("\n[EPOCH: {}], \tTest Loss: {:.4f}, \tTest Accuracy: {:.2f}% \n".format(epoch, valid_loss, valid_accuracy))